In [ ]:
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from gglasso.helper.utils import sparsity, zero_replacement, normalize, log_transform
from gglasso.problem import glasso_problem

from numpy.linalg import matrix_rank

In [ ]:
soil = pd.read_csv('data/soil/soil_116.csv', sep=',', index_col = 0).T
soil.shape

In [ ]:
soil_0 = soil.replace(1, 0)
soil_0_norm = round(soil_0.sum()/89, 2)
soil_0_norm.shape

In [ ]:
gg_lowrank = pd.read_csv("data/soil/gg_lr2.csv", sep=",", header=None)

In [ ]:
gg_rank = matrix_rank(gg_lowrank)
gg_rank

In [ ]:
def robust_PCA(X, L, inverse=True):
    sig, V = np.linalg.eigh(L)
    ind = np.argwhere(sig > 1e-9)

    if inverse:
        loadings = V[:,ind] @ np.diag(np.sqrt(1/sig[ind]))
    else:
        loadings = V[:,ind] @ np.diag(np.sqrt(sig[ind]))

    zu = X.values.T @ loadings
    
    return zu, loadings

In [ ]:
zu_gg, gg_loadings = robust_PCA(X, gg_lowrank, inverse=True)

In [ ]:
zu_gg.shape, soil_0_norm.shape

In [ ]:
for i in range(gg_rank):
    plt.scatter(zu_gg[:,i], soil_0_norm)
    plt.xlabel(f"PCA component {i+1}")
    plt.ylabel("% of non-zero entries")
    plt.savefig('sparsity_soil{0}.png'.format(i+1))
    plt.show()

In [ ]:
for i in range(0, gg_rank):
    print("Spearman correlation between % of non-zero entries and {0}th component: {1}, p-value: {2}".format(i+1, stats.spearmanr(ph, zu_gg[:,i])[0], 
                                                                              stats.spearmanr(ph, zu_gg[:,i])[1]))